In [4]:
import pennylane as qml
from pennylane import numpy as np

## PF.4.1

In [5]:
dev = qml.device("default.qubit", wires = 3)

@qml.qnode(dev)
def circuit_as_function(params):
    """
    Implements the circuit shown in the codercise statement.
    Args:
    - params (np.ndarray): [theta_0, theta_1, theta_2, theta_3]
    Returns:
    - (np.tensor): <Z0>
    """

    ####################
    ###YOUR CODE HERE###
    ####################
    qml.RX(params[0], wires=0)
    qml.CNOT(wires=(0, 1))
    qml.CNOT(wires=(1, 2))
    qml.CNOT(wires=(2, 0))
    qml.RY(params[1], wires=0)
    qml.RY(params[2], wires=1)
    qml.RY(params[3], wires=2)

    return qml.expval(qml.Z(wires=0)) # Return the expectation value

angles = np.linspace(0, 4 * np.pi, 200)
output_values = np.array([circuit_as_function([0.5, t, 0.5, 0.5]) for t in angles])

## PF.4.2

In [11]:
dev = qml.device("default.qubit", wires = 3)

@qml.qnode(dev)
def strong_entangler(params):
    """
    Applies Strongly Entangling Layers to the default initial state
    Args:
    - weights (np.ndarray): The weights argument for qml.StronglyEntanglingLayers
    Returns:
    - (np.tensor): <Z0>
    """

    ####################
    ###YOUR CODE HERE###
    ####################
    qml.StronglyEntanglingLayers(params, wires = range(3))
    
    return qml.expval(qml.PauliZ(0))

test_weights = [[[0.1,0.2,0.3],[0.4,0.5,0.6],[0.7, 0.8, 0.9]],[[0.1,0.1,0.1],[0.3,0.3,0.3],[0.4,0.4,0.4]]] # Write some valid weights here.

print("The output of your circuit with these weights is: ", strong_entangler(test_weights))

The output of your circuit with these weights is:  0.7143314713276365


## PF.4.3

In [ ]:
dev = qml.device("default.qubit", wires = 3)

@qml.qnode(dev)
def embedding_and_circuit(features, params):
    """
    A QNode that depends on trainable and non-trainable parameters
    Args:
    - features (np.ndarray): Non-trainable parameters in the AngleEmbedding routine
    - params (np.ndarray): Trainable parameters for the rest of the circuit
    Returns:
    - (np.tensor): <Z0>
    """

    ####################
    ###YOUR CODE HERE###
    ####################
    qml.AngleEmbedding(features, wires=range(3))
    qml.CNOT(wires=(0, 1))
    qml.CNOT(wires=(1, 2))
    qml.CNOT(wires=(2, 0))
    qml.RY(params[0], wires=0)
    qml.RY(params[1], wires=1)
    qml.RY(params[2], wires=2)
    
    return qml.expval(qml.PauliZ(0))

features = np.array([0.3,0.4,0.6], requires_grad = False)
params = np.array([0.4,0.7,0.9], requires_grad = True)
print("The gradient of the circuit is:", qml.jacobian(embedding_and_circuit)(features, params))


## PF.4.4

In [ ]:
dev = qml.device("default.qubit", wires = 2)

@qml.qnode(dev, diff_method = "parameter-shift", max_diff = 2)
def circuit_for_hessian(params):
    """
    Implements the circuit shown in the codercise statement
    Args:
    - params (np.ndarray): [theta_0, theta_1, theta_2, theta_3]
    Returns:
    - np.tensor: <Z0xZ1>
    """

    ####################
    ###YOUR CODE HERE###
    ####################
    qml.RY(params[0], wires=0)
    qml.IsingXX(params[1], wires=(0, 1))
    qml.RX(params[2], wires=0)
    qml.RX(params[3], wires=1)

    return qml.expval(qml.PauliZ(wires=0) @ qml.PauliZ(wires=1)) # Return the expectation value required

test_params = np.array([0.1,0.2,0.3,0.4], requires_grad = True)
# Don't change test_params! 

hessian = qml.jacobian(qml.jacobian(circuit_for_hessian))(test_params) # Compute the Hessian
print("The hessian of the circuit is: \n", hessian)


## PF.4.5a

In [ ]:
def cost_function(params):
    """
    Computes the cost function given in the codercise, as a function of the
    parameters of circuit_as_function.
    Args:
    - params (np.ndarray): The parameters we pass to circuit_as_function
    Returns:
    - np.float: The cost function evaluated in params.
    """
    ################
    #YOUR CODE HERE#
    ################
    result = circuit_as_function(params)

    return result ** 3 - result ** 2 / 2 + result # Return the value of the cost function


## PF.4.5b

In [ ]:
def optimize(cost_function, init_params, steps):

    opt = qml.GradientDescentOptimizer(stepsize = 0.4) # Change this as you see fit

    params = init_params

    for i in range(steps):

        params = opt.step(cost_function, params)

    return params, cost_function(params)

minimum = optimize(cost_function, np.random.random_sample((4,)), steps=100)[1] # An np.tensor of shape () containing the minimum of cost_function.
